In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

import pandas as pd
#from six.moves import urllib
import tensorflow as tf

In [2]:
import pandas as pd

CSV_COLUMNS = [
    "pressure", "temperature", "wind_speed", "wind_direction", "sunshine", "precipitation_bracket"
]


pressure = tf.feature_column.numeric_column("pressure")
temperature = tf.feature_column.numeric_column("temperature")
wind_speed = tf.feature_column.numeric_column("wind_speed")
wind_direction = tf.feature_column.numeric_column("wind_direction")
sunshine = tf.feature_column.numeric_column("sunshine")

base_columns = [
    pressure, temperature, wind_speed, wind_direction, sunshine
]


crossed_columns = [

]

deep_columns = [
    tf.feature_column.indicator_column(pressure),
    tf.feature_column.indicator_column(temperature),
    tf.feature_column.indicator_column(wind_direction),
    tf.feature_column.indicator_column(wind_speed),
    tf.feature_column.indicator_column(sunshine),
]

#import training and test data from csv
df_train = pd.read_csv("typhoon_train_new1.csv", names=CSV_COLUMNS, skipinitialspace=True, skiprows=1)
df_test = pd.read_csv("typhoon_test_new1.csv", names=CSV_COLUMNS, skipinitialspace=True, skiprows=1)
train_labels = (df_train["precipitation_bracket"].apply(lambda x: "<20cm" in x)).astype(int)
test_labels = (df_test["precipitation_bracket"].apply(lambda x: "<20cm" in x)).astype(int)

In [3]:
def build_estimator(model_dir, model_type):
  """Build an estimator."""
  if model_type == "wide":
    m = tf.estimator.LinearClassifier(
        model_dir=model_dir, feature_columns=base_columns + crossed_columns)
  elif model_type == "deep":
    m = tf.estimator.DNNClassifier(
        model_dir=model_dir,
        feature_columns=deep_columns,
        hidden_units=[100, 50])
  else:
    m = tf.estimator.DNNLinearCombinedClassifier(
        model_dir=model_dir,
        linear_feature_columns=crossed_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=[100, 50])
  return m

In [4]:
def input_fn_train(data_file, num_epochs, shuffle):
    df_data = pd.read_csv(
        "/Users/Kid/Desktop/DeepLearningNg/pynb/typhoon_train_new1.csv",
        names = CSV_COLUMNS,
        skipinitialspace = True,
        engine = "python",
        skiprows = 1
    )
    #remove NaN elements
    df_data = df_data.dropna(how="any", axis=0)
    labels = df_data["precipitation_bracket"].apply(lambda x: "<20cm" in x).astype(int)
    return tf.estimator.inputs.pandas_input_fn(
      x = df_data,
      y = labels,
      batch_size = 10,
      num_epochs=num_epochs,
      shuffle=shuffle,
      num_threads=5)

In [5]:
def input_fn_test(data_file, num_epochs, shuffle):
    df_data = pd.read_csv(
        "/Users/Kid/Desktop/DeepLearningNg/pynb/typhoon_test_new1.csv",
        names = CSV_COLUMNS,
        skipinitialspace = True,
        engine = "python",
        skiprows = 1
    )
    #remove NaN elements
    df_data = df_data.dropna(how="any", axis=0)
    labels = df_data["precipitation_bracket"].apply(lambda x: "<20cm" in x).astype(int)
    return tf.estimator.inputs.pandas_input_fn(
      x = df_data,
      y = labels,
      batch_size = 10,
      num_epochs=num_epochs,
      shuffle=shuffle,
      num_threads=5)

In [6]:
df_data = pd.read_csv(
    "/Users/Kid/Desktop/DeepLearningNg/pynb/typhoon_train_new1.csv",
    names = CSV_COLUMNS,
    skipinitialspace = True,
    engine = "python",
    skiprows = 1
)
df_data


,pressure,temperature,wind_speed,wind_direction,sunshine,precipitation_bracket
0,879.0,21.00,0.3,180.0,0.0,<20cm
1,878.0,20.20,0.1,0.0,0.0,<20cm
2,877.0,20.90,0.0,0.0,0.0,<20cm
3,877.0,20.60,0.0,0.0,0.0,<20cm
4,876.0,20.10,0.8,360.0,0.0,<20cm
5,876.0,20.00,1.1,360.0,0.0,<20cm
6,876.0,19.75,1.0,22.5,0.1,<20cm
7,876.0,19.50,0.8,67.5,0.0,<20cm
8,876.0,19.60,0.0,0.0,0.0,<20cm
9,876.0,20.50,0.0,0.0,0.0,<20cm


In [7]:
def train_and_eval(model_dir, model_type, train_steps, train_data, test_data):
  """Train and evaluate the model."""
  train_file_name, test_file_name = "typhoon_train_new1.csv","typhoon_test_new1.csv"
  model_dir = tempfile.mkdtemp() if not model_dir else model_dir

  m = build_estimator(model_dir, model_type)
  # set num_epochs to None to get infinite stream of data.
  m.train(
      input_fn=input_fn_train(train_file_name, num_epochs=None, shuffle=True),
      steps=train_steps)
  # set steps to None to run evaluation until all data consumed.
  results = m.evaluate(
      input_fn=input_fn_test(test_file_name, num_epochs=1, shuffle=False),
      steps=None)
  print("model directory = %s" % model_dir)
  for key in sorted(results):
    print("%s: %s" % (key, results[key]))

FLAGS = None

def main(_):
  train_and_eval(FLAGS.model_dir, FLAGS.model_type, FLAGS.train_steps,
                 FLAGS.train_data, FLAGS.test_data)

In [8]:
if __name__ == "__main__":
  parser = argparse.ArgumentParser()
  parser.register("type", "bool", lambda v: v.lower() == "true")
  parser.add_argument(
      "--model_dir",
      type=str,
      default="",
      help="Base directory for output models."
  )
  parser.add_argument(
      "--model_type",
      type=str,
      default="wide",
      help="Valid model types: {'wide', 'deep', 'wide_n_deep'}."
  )
  parser.add_argument(
      "--train_steps",
      type=int,
      default=100,
      help="Number of training steps."
  )
  parser.add_argument(
      "--train_data",
      type=str,
      default="/Users/Kid/Desktop/DeepLearningNg/pynb/typhoon_train_new1.csv",
      help="Path to the training data."
  )
  parser.add_argument(
      "--test_data",
      type=str,
      default="/Users/Kid/Desktop/DeepLearningNg/pynb/typhoon_test_new1.csv",
      help="Path to the test data."
  )
  FLAGS, unparsed = parser.parse_known_args()
  print(FLAGS)
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Namespace(model_dir='', model_type='wide', test_data='/Users/Kid/Desktop/DeepLearningNg/pynb/typhoon_test_new1.csv', train_data='/Users/Kid/Desktop/DeepLearningNg/pynb/typhoon_train_new1.csv', train_steps=100)
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/9l/dqxy6vv936v6zx9xk0h54hqw0000gn/T/tmpfg3m1ket', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/9l/dqxy6vv936v6zx9xk0h54hqw0000gn/T/tmpfg3m1ket/model.ckpt.
INFO:tensorflow:loss = 6.93147, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /var/folders/9l/dqxy6vv936v6zx9xk0h54hqw0000gn/T/tmpfg3m1ket/model.ckpt.
INFO:tensorflow:Loss for final step: 28.6944.
INFO:tensorflow:Starting evaluation

SystemExit: 

/Users/Kid/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
